# Billing Configuration & Year Generator Diagnostics

## Overview
This notebook diagnoses two critical issues in the fees-up system:

1. **PowerSync Sync Errors**: Checkpoint failures with "Could not apply checkpoint due to local data" messages
2. **Year/Month Generator**: Seeding is successful locally, but data isn't syncing to the server
3. **Billing Configuration**: Setup and validation of billing settings UI and database integration

## Key Log Findings

### Issue 1: Connection Errors
```
[PowerSync] WARNING: Sync service error
ClientException: Connection closed while receiving data, uri=https://...powersync.journeyapps.com/sync/stream
```
**Root Cause**: Network connectivity timeout or server-side connection issue. PowerSync client is closing the connection prematurely.

### Issue 2: Checkpoint Blocking (CRITICAL)
```
[PowerSync] INFO: Could not apply checkpoint due to local data. Will retry at completed upload or next checkpoint.
(repeated 200+ times)
```
**Root Cause**: The year/month seeding creates local data (192 months × 16 years = 3,072 records), but PowerSync's `uploadData()` is not being called. The checkpoint processor sees pending local data and blocks synchronization until the data is uploaded.

### Issue 3: Missing school_id
```
⚠️ No school_id in auth metadata. Security rules will pull on next login.
```
**Root Cause**: The authenticated user doesn't have `school_id` in their JWT metadata. This may prevent proper row-level security filtering.

## Solution Strategy

### For Year Generator Seeding:
1. Add explicit `uploadData()` call after seeding completes
2. Use `batch` mode to upload all 3,072+ records efficiently
3. Add error handling and retry logic
4. Verify Supabase `school_years` and `school_year_months` tables have proper RLS policies

### For Billing Config:
1. Ensure UI is wired to actual database provider
2. Validate CRUD operations for `billing_configurations` table
3. Test real-time updates via Riverpod providers

## Section 1: Current Year Generator Implementation

### File: `lib/data/providers/school_year_generator_provider.dart`

**Status**: ✅ Seeding works locally (generates 3,072 records)  
**Issue**: ❌ No upload to server after seeding completes

**Current Flow**:
```
1. schoolYearSeederProvider triggered
2. Generates 16 years (2020-2036)
3. For each year, creates 12 months
4. Uses writeTransaction() for batch efficiency
5. Logs "✅ Year/Month Seeding Complete"
6. ❌ MISSING: uploadData() call to sync with server
```

**Problem**: PowerSync's checkpoint mechanism is blocking because:
- Local data exists in SQLite (school_years, school_year_months)
- No explicit `uploadData()` issued to Supabase
- PowerSync's upload queue remains non-empty
- Checkpoint processor waits indefinitely, causing 200+ "Could not apply checkpoint" messages

## Section 2: Billing Configuration Status

### File: `lib/pc/widgets/settings/billing_config_card.dart`

**Current State**: 
- ✅ UI renders properly
- ✅ Form fields present (currency, tax rate, fees, invoice settings)
- ⚠️ State management using local TextEditingControllers
- ❌ No real-time sync with `billingConfigProvider`

**Form Fields**:
1. Base Currency (dropdown)
2. Default Tax Rate (%)
3. Annual Tuition Fee
4. Registration Fee
5. Late Fee
6. Grace Days
7. Invoice Prefix
8. Invoice Sequence
9. Invoice Footer
10. Allow Partial Payments (toggle)

**Missing Integrations**:
1. No `_hydrateIfNeeded()` implementation for updating UI from DB
2. No `_onSavePressed()` handler to persist changes
3. No real-time updates when provider data changes
4. No validation before save

### Provider: `lib/data/providers/billing_config_provider.dart`

**Expected Data Structure**:
```dart
Map<String, dynamic> {
  'id': String,
  'school_id': String,
  'base_currency': String,        // e.g., "USD"
  'tax_rate': double,              // e.g., 0.08
  'default_fee': double,           // Tuition
  'registration_fee': double,
  'late_fee': double,
  'grace_days': int,
  'invoice_prefix': String,        // e.g., "INV-"
  'invoice_sequence': int,         // Current sequence number
  'invoice_footer': String,        // Custom text
  'allow_partial_payments': bool,
  'created_at': String,
  'updated_at': String,
}
```

## Section 3: Action Plan - Year Generator Fix

### Step 1: Update `school_year_generator_provider.dart`

**Change Required**: Add explicit `uploadData()` call after seeding

**Why**: PowerSync only syncs data if:
1. It's flagged as "local" in SQLite
2. An explicit `uploadData()` call is issued
3. The Supabase connector processes the upload

**Implementation Strategy**:
```
1. After writeTransaction() completes
2. Catch any database errors
3. Call dbService.db.uploadData() with retry logic
4. Wait for upload to complete (max 30 seconds)
5. Log success/failure
6. If fails, log which school_id to manually troubleshoot
```

### Step 2: Validate Database Tables

**Checklist**:
- [ ] `school_years` table exists in Supabase
- [ ] `school_year_months` table exists in Supabase
- [ ] RLS policies allow school-scoped writes
- [ ] Foreign key constraints are properly configured
- [ ] `school_id` column is indexed for performance

### Step 3: Fix PowerSync Connector

**File**: `lib/data/services/supabase_connector.dart`

**Action**: Ensure the connector's `uploadData()` method:
- Handles batch uploads efficiently
- Retries on network errors
- Properly marks records as synced after success
- Doesn't crash on partial failures

## Section 4: Action Plan - Billing Configuration Implementation

### Task 1: Complete BillingConfigCard Implementation

**Missing Methods to Implement**:

1. `_hydrateIfNeeded(Map<String, dynamic> config)`:
   - Check if controllers are empty
   - Populate with config values from database
   - Mark `_hydrated = true` to prevent re-hydration

2. `_onSavePressed()`:
   - Validate all form fields
   - Show loading indicator
   - Call provider's save method
   - Handle success/error states
   - Clear state on success

3. `_buildInput()` Improvements:
   - Add proper error handling for numeric fields
   - Add validation callbacks
   - Add clear helper text for complex fields

### Task 2: Ensure Provider Is Wired Correctly

**Provider Location**: `lib/data/providers/billing_config_provider.dart`

**Implementation Checklist**:
- [ ] Reads from `billing_configurations` table
- [ ] Filtered by `school_id` from `dashboardDataProvider`
- [ ] Returns real-time updates via FutureProvider or StreamProvider
- [ ] Has save/update methods for CRUD operations
- [ ] Handles errors gracefully with fallback defaults

### Task 3: Test Billing Configuration Workflow

**Test Scenarios**:
1. Load BillingConfigCard for a school
2. Verify form fields populate with database values
3. Modify a value (e.g., tax rate from 8% to 10%)
4. Click Save
5. Verify database is updated
6. Reload screen to confirm persistence
7. Check that other users see the updated value in real-time

## Summary: Execution Order

### Phase 1: Fix Year Generator (FIRST PRIORITY)
**Expected Time**: 30-45 minutes

1. ✅ Diagnose: Understand checkpoint blocking issue
2. 🔧 Fix: Add `uploadData()` call to year seeder
3. ✅ Test: Verify sync completes, checkpoint succeeds
4. ✅ Validate: Confirm all 192 months appear in Supabase

**Success Criteria**:
- Year/month seeding completes
- PowerSync checkpoint succeeds (no "Could not apply" messages)
- Database shows all 16 years × 12 months = 192 records
- No sync errors in logs

### Phase 2: Fix Billing Configuration (SECOND PRIORITY)
**Expected Time**: 60-90 minutes

1. ✅ Complete `BillingConfigCard` implementation
2. ✅ Ensure `billingConfigProvider` is wired correctly
3. ✅ Add hydration logic to populate form from DB
4. ✅ Add save/persist functionality
5. ✅ Test CRUD operations

**Success Criteria**:
- Form loads with current billing config values
- Changes persist to database
- Updates reflect in real-time for other users
- No errors when saving

### Phase 3: Integration Testing (THIRD PRIORITY)
**Expected Time**: 30-45 minutes

1. ✅ Test billing config in student creation workflow
2. ✅ Verify year/month selection works in student dialog
3. ✅ Confirm invoices use correct settings
4. ✅ Test edge cases (no config, missing years, etc.)

**Success Criteria**:
- Student dialog correctly pulls active year/month
- Invoices generate with configured fees
- No errors during real-world usage